In [1]:
print("Jesus botado")

Jesus botado


In [2]:
from ultralytics import YOLO

# Load a pretrained YOLO model
model = YOLO("yolo11n.pt")

# Perform object detection on an image
results = model("https://ultralytics.com/images/bus.jpg")

# Visualize the results
for result in results:
    result.show()


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 d:\repos\Phone detection\bus.jpg: 640x480 4 persons, 1 bus, 242.9ms
Speed: 4.6ms preprocess, 242.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 480)


In [3]:
# Perform object detection on an image
results = model("test.png")

# Visualize the results
for result in results:
    result.show()


image 1/1 d:\repos\Phone detection\test.png: 352x640 5 persons, 4 chairs, 1 dining table, 1 refrigerator, 153.9ms
Speed: 3.0ms preprocess, 153.9ms inference, 4.0ms postprocess per image at shape (1, 3, 352, 640)


Creating the dataset

In [5]:
import pandas as pd
import os
from PIL import Image

def convert_to_yolo_format(csv_file, image_folder, output_folder, class_mapping):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Create output folder if doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    for _, row in df.iterrows():
        # Get image filename and bounding box coordinates
        image_filename = row['filename']  # Assuming the CSV has an 'image' column with image names
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        class_name = row['class']  # Assuming 'class' column contains class name or label
        image_width = row['width']
        image_height = row['height']

        # Get image dimensions
        #image_path = os.path.join(image_folder, image_filename)
        #image = Image.open(image_path)
        #image_width, image_height = image.size

        
        # Normalize the bounding box to YOLO format
        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        # Get the class ID from the mapping
        class_id = class_mapping.get(class_name, -1)

        # Prepare YOLO format annotation line
        yolo_annotation = f"{class_id} {x_center} {y_center} {width} {height}\n"

        # Write the annotation to a file
        output_txt_path = os.path.join(output_folder, os.path.splitext(image_filename)[0] + ".txt")
        with open(output_txt_path, 'a') as f:
            f.write(yolo_annotation)

# Define class mappings (if class names are used)
class_mapping = {
    'Comp': 0
    # Add more classes as needed
}

# Example usage
csv_file = 'dataset_ori/labels.csv'  # Path to your CSV file
image_folder = 'dataset_ori/positive/'       # Folder where images are stored
output_folder = 'dataset_ori/yolo_annotations_v2/'  # Folder to save YOLO annotations

convert_to_yolo_format(csv_file, image_folder, output_folder, class_mapping)


In [7]:
image_folder = 'dataset_ori/negative/'
output_folder = 'dataset_ori/yolo_annotations_v3/'  # Folder to save YOLO annotations
# Create output folder if doesn't exist
os.makedirs(output_folder, exist_ok=True)
for image_filename in os.listdir(image_folder):
    # Write the annotation to a file
    output_txt_path = os.path.join(output_folder, os.path.splitext(image_filename)[0] + ".txt")
    with open(output_txt_path, 'a') as f:
        f.write('')

In [9]:
import os
import shutil
import random

# Define paths
folder1 = 'dataset_ori/positive'
folder2 = 'dataset_ori/negative'
folder3 = 'dataset_ori/yolo_labels'
train_dir = 'dataset/train/images'
train_label = 'dataset/train/labels'
val_dir = 'dataset/val/images'
val_label = 'dataset/val/labels'

# Create train/val directories if they don't exist
#os.makedirs(os.path.join(train_dir, 'folder1'), exist_ok=True)
#os.makedirs(os.path.join(val_dir, 'folder1'), exist_ok=True)
#os.makedirs(os.path.join(train_dir, 'folder2'), exist_ok=True)
#os.makedirs(os.path.join(val_dir, 'folder2'), exist_ok=True)

# Function to split images into train/val split
def split_images(source_folder, labels_folder, train_folder, train_label, val_folder, val_label, split_ratio=0.8):
    # List all images in the source folder
    all_images = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    all_labels = [f for f in os.listdir(labels_folder) if os.path.isfile(os.path.join(labels_folder, f))]
    
    # Shuffle the images for randomness
    random.shuffle(all_images)

    # Calculate the number of training images
    split_index = int(len(all_images) * split_ratio)

    # Split the images into train and validation
    train_images = all_images[:split_index]
    val_images = all_images[split_index:]

    # Move the images to the corresponding folders
    for img in train_images:
        shutil.copy(os.path.join(source_folder, img), os.path.join(train_folder, img))
        name = img.split('.')[0] + '.txt'
        if name in all_labels:
            shutil.copy(os.path.join(labels_folder, name), os.path.join(train_label, name))
        else:
            print(name+" no encontrado en labels")

    for img in val_images:
        shutil.copy(os.path.join(source_folder, img), os.path.join(val_folder, img))
        name = img.split('.')[0] + '.txt'
        if name in all_labels:
            shutil.copy(os.path.join(labels_folder, name), os.path.join(val_label, name))
        else:
            print(name+" no encontrado en labels")

# Split images from both folders
split_images(folder1, folder3, train_dir, train_label, val_dir, val_label)
split_images(folder2, folder3, train_dir, train_label, val_dir, val_label)

print("Images have been split into train and validation sets.")


Images have been split into train and validation sets.


In [11]:
from ultralytics import YOLO

# Load a pretrained YOLO model
model = YOLO("yolo11n.pt")

# Train the model
results = model.train(
    data="cellphone_dataset.yaml",  # Path to the dataset configuration file
    epochs=50,                      # Number of training epochs
    #imgsz=640,                      # Image size
    batch=16,                       # Batch size
    name="yolov8_cellphone"         # Name of the training run
)

New https://pypi.org/project/ultralytics/8.3.71 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.70  Python-3.10.4 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=cellphone_dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_cellphone2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

100%|██████████| 755k/755k [00:00<00:00, 5.18MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning D:\repos\Phone detection\dataset\train\labels... 710 images, 429 backgrounds, 0 corrupt: 100%|██████████| 861/861 [00:02<00:00, 334.76it/s]

train: New cache created: D:\repos\Phone detection\dataset\train\labels.cache


...

      50/50         0G       1.16     0.9352      1.235          9        640: 100%|██████████| 54/54 [05:39<00:00,  6.28s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:41<00:00,  3.78s/it]

                   all        329        129      0.328      0.527      0.234      0.105



50 epochs completed in 5.361 hours.
Optimizer stripped from runs\detect\yolov8_cellphone2\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\yolov8_cellphone2\weights\best.pt, 5.4MB

Validating runs\detect\yolov8_cellphone2\weights\best.pt...
Ultralytics 8.3.70  Python-3.10.4 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:36<00:00,  3.29s/it]


                   all        329        129      0.335      0.593      0.259      0.112
Speed: 0.8ms preprocess, 58.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\yolov8_cellphone2


In [12]:
# Evaluate the model
metrics = model.val()  # Evaluate on the validation set
print(metrics)         # Print evaluation metrics (mAP, precision, recall, etc.)

Ultralytics 8.3.70  Python-3.10.4 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\repos\Phone detection\dataset\val\labels.cache... 178 images, 221 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:39<00:00,  1.87s/it]


                   all        329        129      0.337      0.574       0.26      0.112
Speed: 0.8ms preprocess, 53.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\yolov8_cellphone22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002055873AE30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0

In [4]:
# Load the trained model
model_trained = YOLO("runs/detect/yolov8_cellphone2/weights/best.pt")

# Run inference on an image
results = model_trained("test.png")

# Visualize the results
for result in results:
    result.show()


image 1/1 d:\repos\Phone detection\test.png: 352x640 (no detections), 150.2ms
Speed: 3.1ms preprocess, 150.2ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)


In [5]:
import cv2
# Path to the input video
video_path = "0122_cut.mp4"

# Run inference on the video
results = model(video_path, stream=True)  # 'stream=True' processes the video frame by frame

# Loop through the results and display/save the output
for result in results:
    # Visualize the results on the frame
    annotated_frame = result.plot()

    # Display the annotated frame
    cv2.imshow("YOLOv8 Inference", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cv2.destroyAllWindows()


video 1/1 (frame 1/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 164.3ms
video 1/1 (frame 2/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 160.0ms
video 1/1 (frame 3/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 149.5ms
video 1/1 (frame 4/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 109.7ms
video 1/1 (frame 5/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 128.7ms
video 1/1 (frame 6/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 118.9ms
video 1/1 (frame 7/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 147.8ms
video 1/1 (frame 8/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 114.4ms
video 1/1 (frame 9/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 159.9ms
video 1/1 (frame 10/1489) d:\repos\Phone detection\0122_cut.mp4: 384x640 (no detections), 145.6ms
video 1/1 (frame 11/1489) d:

In [5]:
# Run inference on an image
results = model_trained("aver.jpg")

# Visualize the results
for result in results:
    result.show()


image 1/1 d:\repos\Phone detection\aver.jpg: 416x640 2 cellphone_uses, 211.9ms
Speed: 5.5ms preprocess, 211.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)


In [9]:
import os
for img in os.listdir('test_images/'):
    # Run inference on an image
    results = model_trained(os.path.join('test_images/',img))

    # Visualize the results
    for result in results:
        result.show()


image 1/1 d:\repos\Phone detection\test_images\aver.jpg: 416x640 2 cellphone_uses, 203.2ms
Speed: 3.5ms preprocess, 203.2ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 d:\repos\Phone detection\test_images\test_2.jpg: 352x640 (no detections), 163.1ms
Speed: 1.0ms preprocess, 163.1ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 d:\repos\Phone detection\test_images\test_3.png: 384x640 2 cellphone_uses, 154.6ms
Speed: 3.0ms preprocess, 154.6ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 d:\repos\Phone detection\test_images\test_4.jpg: 448x640 2 cellphone_uses, 156.9ms
Speed: 4.0ms preprocess, 156.9ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 d:\repos\Phone detection\test_images\test_copy.png: 640x384 (no detections), 135.5ms
Speed: 2.5ms preprocess, 135.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


In [11]:
# Load a pretrained YOLO model
new_model = YOLO("yolo11m-cls.pt")

# Perform object detection on an image
results = new_model("test.png")

# Visualize the results
for result in results:
    result.show()


image 1/1 d:\repos\Phone detection\test.png: 224x224 lab_coat 0.51, library 0.06, dishwasher 0.03, barbershop 0.03, shoe_shop 0.03, 105.0ms
Speed: 36.4ms preprocess, 105.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [14]:
for img in os.listdir('test_images/'):
    # Run inference on an image
    results = model_trained(os.path.join('test_images/',img))

    # Visualize the results
    for result in results:
        result.show()


image 1/1 d:\repos\Phone detection\test_images\aver.jpg: 416x640 2 cellphone_uses, 177.2ms
Speed: 0.0ms preprocess, 177.2ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 d:\repos\Phone detection\test_images\test_2.jpg: 352x640 (no detections), 138.7ms
Speed: 3.0ms preprocess, 138.7ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 d:\repos\Phone detection\test_images\test_3.png: 384x640 2 cellphone_uses, 150.5ms
Speed: 0.0ms preprocess, 150.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 d:\repos\Phone detection\test_images\test_4.jpg: 448x640 2 cellphone_uses, 152.3ms
Speed: 0.0ms preprocess, 152.3ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 d:\repos\Phone detection\test_images\test_copy.png: 640x384 (no detections), 142.6ms
Speed: 0.0ms preprocess, 142.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


In [15]:
# Load a pretrained YOLO model
last_model = YOLO("yolo11l.pt")

# Perform object detection on an image
results = last_model("test.png")

# Visualize the results
for result in results:
    result.show()

100%|██████████| 49.0M/49.0M [00:41<00:00, 1.23MB/s]



image 1/1 d:\repos\Phone detection\test.png: 352x640 6 persons, 1 backpack, 7 chairs, 2 dining tables, 1 laptop, 1 refrigerator, 640.2ms
Speed: 2.0ms preprocess, 640.2ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
